In [2]:
# -----------------------------
# Mount Google Drive
# -----------------------------
from google.colab import drive
drive.mount('/content/drive')

# -----------------------------
# Imports
# -----------------------------
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from torch.utils.data import Dataset
import torch
import math
import os

# -----------------------------
# Paths
# -----------------------------
output_dir = "/content/drive/MyDrive/gpt2-fairytales-medium"  # your saved model
cleaned_dataset_file = "fairy_tales_cleaned.txt"              # your cleaned dataset

# -----------------------------
# Load saved model & tokenizer
# -----------------------------
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(output_dir)

# -----------------------------
# Load cleaned dataset (first 500 lines only)
# -----------------------------
with open(cleaned_dataset_file, "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

subset_lines = lines[:500]  # take first 500 stories

# -----------------------------
# Dataset class
# -----------------------------
class StoryDataset(Dataset):
    def __init__(self, texts, tokenizer, block_size=256):
        self.examples = []
        for story in texts:
            tokenized = tokenizer(
                story,
                truncation=True,
                max_length=block_size,
                padding="max_length",
                return_tensors="pt"
            )
            self.examples.append(tokenized["input_ids"].squeeze())

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return self.examples[i]

eval_dataset = StoryDataset(subset_lines, tokenizer)

# -----------------------------
# Data collator (for language modeling)
# -----------------------------
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# -----------------------------
# Temporary Trainer just for evaluation
# -----------------------------
training_args = TrainingArguments(
    output_dir="./temp",
    per_device_eval_batch_size=8,  # larger batch to speed up evaluation
    fp16=True                      # mixed precision for faster evaluation on GPU
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# -----------------------------
# Evaluate Perplexity
# -----------------------------
results = trainer.evaluate()
perplexity = math.exp(results["eval_loss"])
print("Perplexity on first 500 stories:", perplexity)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-717225347.py:75: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nipunikumudika (nipunikumudika-home) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Perplexity on first 500 stories: 1.7614142689378363


In [5]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00


In [7]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9c6b580e62a6d14578bfb949ab7f7f7e75cc2c71fda2d567d554771907a96c8c
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [14]:



reference_story = "Once there lived a certain king who understood the language of all birds and beasts and insects. This knowledge had of course been given him by a fairy godmother; but it was rather a troublesome present, for he knew that if he were ever to reveal anything he had thus learned he would turn into a stone. How he managed to avoid doing so long before this story opens I cannot say, but he had safely grown up to manhood, and married a wife, and was as happy as monarchs generally are.  This king, I must tell you, was a Hindu; and when a Hindu eats his food he has a nice little place on the ground freshly plastered with mud, and he sits in the middle of it with very few clothes on--which is quite a different way from ours.  Well, one day the king was eating his dinner in just such a nice, clean, mud-plastered spot, and his wife was sitting opposite to wait upon him and keep him company. As he ate he dropped some grains of rice upon the ground, and a little ant, who was running about seeking a living, seized upon one of the grains and bore it off towards his hole. Just outside the king's circle this ant met another ant, and the king heard the second one say:  'Oh, dear friend, do give me that grain of rice, and get another one for yourself. You see my boots are so dirty that, if I were to go upon the king's eating place, I should defile it, and I can't do that, it would be so very rude.'  But the owner of the grain of rice only replied:  'If you want rice go and get it. No one will notice your dirty boots; and you don't suppose that I am going to carry rice for all our kindred?'  Then the king laughed.  The queen looked at herself up and down, but she could not see or feel anything in her appearance to make the king laugh, so she said:  'What are you laughing at?'  'Did I laugh?' replied the king.  'Of course you did,' retorted the queen; 'and if you think that I am ridiculous I wish you would say so, instead of behaving in that stupid way! What are you laughing at?'  'I'm not laughing at anything,' answered the king.  'Very well, but you _did_ laugh, and I want to know why.'  'Well, I'm afraid I can't tell you,' said the king.  'You _must_ tell me,' replied the queen impatiently. 'If you laugh when there's nothing to laugh at you must be ill or mad. What is the matter?'  Still the king refused to say, and still the queen declared that she must and would know. For days the quarrel went on, and the queen gave her husband no rest, until at last the poor man was almost out of his wits, and thought that, as life had become for him hardly worth living while this went on, he might as well tell her the secret and take the consequences.  'But,' thought he, 'if I am to become a stone, I am not going to lie, if I can help it, on some dusty highway, to be kicked here and there by man and beast, flung at dogs, be used as the plaything of naughty children, and become generally restless and miserable. I will be a stone at the bottom of the cool river, and roll gently about there until I find some secure resting-place where I can stay for ever.'  So he told his wife that if she would ride with him to the middle of the river he would tell her what he had laughed at. She thought he was joking, and laughingly agreed; their horses were ordered and they set out.      On the way they came to a fine well beneath the shade of some lofty, wide-spreading trees, and the king proposed that they should get off and rest a little, drink some of the cool water, and then pass on. To this the queen consented; so they dismounted and sat down in the shade by the well-side to rest.  It happened that an old goat and his wife were browsing in the neighbourhood, and, as the king and queen sat there, the nanny goat came to the well's brink and peering over saw some lovely green leaves that sprang in tender shoots out of the side of the well.  'Oh!' cried she to her husband, 'come quickly and look. Here are some leaves which make my mouth water; come and get them for me!'  Then the billy goat sauntered up and looked over, and after that he eyed his wife a little crossly.  'You expect me to get you those leaves, do you? I suppose you don't consider how in the world I am to reach them? You don't seem to think at all; if you did you would know that if I tried to reach those leaves I should fall into the well and be drowned!'  'Oh,' cried the nanny goat, 'why should you fall in? Do try and get them!'  'I am not going to be so silly,' replied the billy goat.  But the nanny goat still wept and entreated.  'Look here,' said her husband, 'there are plenty of fools in the world, but I am not one of them. This silly king here, because he can't cure his wife of asking questions, is going to throw his life away. But I know how to cure you of your follies, and I'm going to.'  And with that he butted the nanny goat so severely that in two minutes she was submissively feeding somewhere else, and had made up her mind that the leaves in the well were not worth having.  Then the king, who had understood every word, laughed once more.  The queen looked at him suspiciously, but the king got up and walked across to where she sat.  'Are you still determined to find out what I was laughing at the other day?' he asked.  'Quite,' answered the queen angrily.  'Because,' said the king, tapping his leg with his riding whip, 'I've made up my mind not to tell you, and moreover, I have made up my mind to stop you mentioning the subject any more.'  'What _do_ you mean?' asked the queen nervously.  'Well,' replied the king, 'I notice that if that goat is displeased with his wife, he just butts her, and that seems to settle the question----'  'Do you mean to say you would _beat_ me?' cried the queen.  'I should be extremely sorry to have to do so,' replied the king; 'but I have got to persuade you to go home quietly, and to ask no more silly questions when I say I cannot answer them. Of course, if you _will_ persist, why----'  And the queen went home, and so did the king; and it is said that they are both happier and wiser than ever before.  (Punjâbi Story, Major Campbell, Feroshepore) <|endofstory|>"

In [16]:
generated_story = """Once there lived a certain king who understood the language of all birdsAn old woman sat at her kitchen table, stirring curds and making custards with some fresh eggs and a potful of butter broth. Suddenly she saw something shining come dancing down from above her. It leaped about two feet high and made such crackling sounds as sent shivers up one's spines. "Oh! Hoity-toity!" shrieked Maie, jumping up and hurrying downstairs. "It is a falcon! See what it does!" Quickly setting the dishes down on another dish covered her head, Miëln left off washing the curds, put them back in their places without upsetting them (for which reason so many delighted Mies obtained good repute as well), stood up and asked the bird why it had done such amazing tricks, went aloft like someone soaring over earth, whiskers erect, plucked fruit out once ready laid; then swooping again home settled firmly in place like swallows, devoured everything under grass, nestled securely against trees and never stirred again until evening came round--always returning unharmed to the same spot whence it had come down or swoopingly scampered overhead. This wonderful bird, called Merlin, had three faces, each half-moon shaped and half serpentine. His body was entirely made only by magic, invisible hands, feet and wings having been added after diligent study. He gained admittance into the King Henry's court chiefly because of these talons, not because any prince liked him better than others. Yet those princes were often deceived by the mere sound sometimes given off by this mysterious being, for they followed whatever he said regardless if it proved to be a lie or truth. With regard always to the princesses whom he courted, though every suitor wisely observed precautions to prevent either false reports arising among them whatsoever nor false reports concerning their secret lovers being spread abroad, yet several months passed before anything could have happened otherwise than what might reasonably Have been expected.[14] When a girl became pregnant during this time a special agent of Habsburg China visited the royal castle frequently, giving orders that none should approach the proud monarch except strictly invited guests, bearing valid documents from the sovereign's minister. In due time a young Princess Mary came of age, attended almost her whole schooling in the West Indies,[15 ] bought a magnificent carriage, mounted it, took leave amid heralds and messengers, strolled forth upon five splendid stables, drove her horses homeward and presented her jewels and a pair petticoats studded"""

In [17]:
import evaluate

# Load metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

# BLEU expects strings, not tokenized lists
bleu_score = bleu.compute(predictions=[generated_story], references=[reference_story])

# ROUGE also expects strings
rouge_score = rouge.compute(predictions=[generated_story], references=[reference_story])

print("Generated Story:\n", generated_story)
print("\nReference Story:\n", reference_story[:500], "...")  # print first 500 chars
print("\nBLEU Score:", bleu_score)
print("ROUGE Score:", rouge_score)


Generated Story:
 Once there lived a certain king who understood the language of all birdsAn old woman sat at her kitchen table, stirring curds and making custards with some fresh eggs and a potful of butter broth. Suddenly she saw something shining come dancing down from above her. It leaped about two feet high and made such crackling sounds as sent shivers up one's spines. "Oh! Hoity-toity!" shrieked Maie, jumping up and hurrying downstairs. "It is a falcon! See what it does!" Quickly setting the dishes down on another dish covered her head, Miëln left off washing the curds, put them back in their places without upsetting them (for which reason so many delighted Mies obtained good repute as well), stood up and asked the bird why it had done such amazing tricks, went aloft like someone soaring over earth, whiskers erect, plucked fruit out once ready laid; then swooping again home settled firmly in place like swallows, devoured everything under grass, nestled securely against trees and